# Real-Time Sign Language Detection

This notebook implements real-time detection using the trained YOLO model.

In [ ]:
import sys
import os
sys.path.append('../')  # Add parent directory to path

from src.models.yolo_detector import YOLOSignDetector
import cv2
import json

In [ ]:
# Set up paths
BASE_DIR = os.path.abspath('../')
MODELS_DIR = os.path.join(BASE_DIR, 'data', 'models')
MODEL_PATH = os.path.join(MODELS_DIR, 'sign_language_yolo', 'weights', 'best.pt')
CLASS_NAMES_PATH = os.path.join(MODELS_DIR, 'class_names.json')

# Initialize detector
detector = YOLOSignDetector()
detector.load_model(MODEL_PATH, CLASS_NAMES_PATH)

print(f"Model loaded from: {MODEL_PATH}")
print(f"Classes: {detector.class_names}")

In [ ]:
# Real-time detection function
def real_time_detection(detector, confidence_threshold=0.5):
    """Run real-time sign language detection"""
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    print("Starting real-time detection. Press 'q' to quit.")
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run inference
            results = detector.model(frame)
            
            # Draw predictions
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                        conf = box.conf[0].cpu().numpy()
                        cls = int(box.cls[0].cpu().numpy())
                        
                        if conf > confidence_threshold:
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            label = f"{detector.class_names[cls]}: {conf:.2f}"
                            cv2.putText(frame, label, (x1, y1-10), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Display frame
            cv2.imshow('Sign Language Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Run real-time detection
# real_time_detection(detector)  # Uncomment to run